In [ ]:
import tkinter as tk
from tkinter import messagebox
from collections import Counter
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd
import numpy as np

# Define colors for the GUI elements
bg_color = "#f0f0f0"
fg_color = "#333333"
highlight_color = "#add8e6" 
success_color = "#66bb6a"
error_color = "#f44336"
button_color = "#0d47a1" 
button_text_color = "#ffffff"
input_error_color = "#ffcccc"

# Global variables
dynamic_entries = {}
n_values = []
bold_font = ("Helvetica", 10, "bold")

# Function to create dynamic input fields based on the number of tracks (S) and spaces per track (H)
def create_dynamic_inputs(T, S):
    for widget in dynamic_frame.winfo_children():
        widget.destroy()  # Clear previous inputs
    
    dynamic_entries.clear()  # Clear the dictionary storing dynamic entries
    
# Create input fields for each track and space
    for i in range(T):
        for j in range(S):
            entry = tk.Entry(dynamic_frame, width=5)
            entry.grid(row=T-i-1, column=S-j-1, padx=5, pady=5)  # Position the entry field
            dynamic_entries[(i, j)] = entry  # Store the entry in the dictionary
            
# Label each track
        tk.Label(dynamic_frame, text=f"Track {i+1}", font=bold_font, bg=bg_color, fg=fg_color).grid(row=T-i-1, column=S, padx=5, pady=5)
    
# Label each space number
    for j in range(S):
        tk.Label(dynamic_frame, text=f"Slot {j+1}", font=bold_font, bg=bg_color, fg=fg_color).grid(row=T, column=S-j-1, padx=5, pady=5)
    
# Add the Done button to finish input
    done_button = tk.Button(dynamic_frame, text="Track the number of wagons per block", font=bold_font, command=lambda: count_e_values(T, S), bg=button_color, fg=button_text_color)
    done_button.grid(row=T + 1, columnspan=S + 1, pady=10)

def count_e_values(T, S):
    global e_values
    e_values = Counter()

# Copy entry values before destroying the widgets
    entries_copy = {k: v.get() for k, v in dynamic_entries.items()}

    for i in range(T):
        j = 0
        while j < S:
            if (i, j) in entries_copy:
                b_value = entries_copy[(i, j)]
                if b_value:
                    k = j + 1
                    while k < S and (i, k) in entries_copy and entries_copy[(i, k)] == b_value:
                        k += 1
                    e_values[b_value] += k - j
                    j = k
                else:
                    j += 1
            else:
                j += 1
    
    display_e_values(e_values, T, S, entries_copy)
    
# Function to track number of wagons in each block
def display_e_values(e_values, T, S, entries_copy):
    for widget in dynamic_frame.winfo_children():
        widget.destroy()  # Clear previous inputs

    row = 0
    column = 0
    for b_value, count in e_values.items():
        label = tk.Label(dynamic_frame, text=f"e[{b_value}]: {count}", bg=bg_color, fg=fg_color)
        label.grid(row=row, column=column, padx=5, pady=5)
        column += 1
        if column == 4:
            column = 0
            row += 1

    done_button = tk.Button(dynamic_frame, text="Clustering of Consecutive Wagons", font=bold_font, command=lambda: cluster_wagons(T, S, entries_copy), bg=button_color, fg=button_text_color)
    done_button.grid(row=row + 1, columnspan=4, pady=10)

# Function to cluster wagons on the right side
def cluster_wagons(T, S, entries_copy):
    global clustered_matrix
    clustered_matrix = [[""] * S for _ in range(T)]  # Define clustered matrix globally

    for i in range(T):
        clustered = []
        j = 0
        while j < S:
            if (i, j) in entries_copy:
                b_value = entries_copy[(i, j)]
                if b_value:
                    clustered.append((j, b_value))
                    k = j + 1
                    while k < S and (i, k) in entries_copy and entries_copy[(i, k)] == b_value:
                        k += 1
                    j = k
                else:
                    j += 1
            else:
                j += 1
        
# Populate the clustered matrix from left to right
        col_idx = 0
        for idx, value in clustered:
            clustered_matrix[i][col_idx] = value
            col_idx += 1

    show_clustered_matrix(clustered_matrix, T, S)
    
# Function to show the clustered matrix
def show_clustered_matrix(clustered_matrix, T, S):
    for widget in dynamic_frame.winfo_children():
        widget.destroy()  # Clear previous inputs
    
    for i in range(len(clustered_matrix)):
        for j in range(len(clustered_matrix[i])):
            value = clustered_matrix[i][j]
            label = tk.Label(dynamic_frame, text=f"{value}" if value is not None else "", borderwidth=2, relief="solid", width=3, height=1)
            label.grid(row=len(clustered_matrix)-i-1, column=len(clustered_matrix[i])-j-1, padx=5, pady=5)
        tk.Label(dynamic_frame, text=f"Track {i+1}", bg=bg_color, fg=fg_color).grid(row=len(clustered_matrix)-i-1, column=S, padx=5, pady=5)
    
    for j in range(len(clustered_matrix[0])):
        tk.Label(dynamic_frame, text=f"Slot {j+1}", bg=bg_color, fg=fg_color).grid(row=len(clustered_matrix), column=S-j-1, padx=5, pady=5)

    done_button = tk.Button(dynamic_frame, text="Track the Number of Blocks Value",font=bold_font, command=lambda: initialize_configuration(clustered_matrix, T, S), bg=button_color, fg=button_text_color)
    done_button.grid(row=len(clustered_matrix) + 1, columnspan=len(clustered_matrix[0]) + 1, pady=10)

# Function to initialize the configuration based on clustered matrix
def initialize_configuration(clustered_matrix, T, S):
    for widget in dynamic_frame.winfo_children():
        widget.grid_remove()  # Hide the widgets
    dynamic_frame.grid_remove()  # Hide the frame

    update_b_value(clustered_matrix)  # Update the p values based on input

# Function to update b values and count the occurrences
def update_b_value(clustered_matrix):
    unique_b_values = set()  # Set to store unique p values
    b_counts = Counter()  # Counter to store occurrences of each b value
    
    for i in range(len(clustered_matrix)):
        for j in range(len(clustered_matrix[i])):
            if clustered_matrix[i][j] != "":
           # if clustered_matrix[i][j] is not None:
                b_value = clustered_matrix[i][j]
                unique_b_values.add(b_value)
                b_counts[b_value] += 1

    b_entry.delete(0, tk.END)  # Clear the b_entry field
    b_entry.insert(0, len(unique_b_values))  # Insert the number of unique b values

# Display the n[k] values
    for widget in n_frame.winfo_children():
        widget.destroy()  # Clear previous n[k] labels

    row = 0
    column = 0
    for k in sorted(b_counts):
        label = tk.Label(n_frame, text=f"n[{k}]: {b_counts[k]}", bg=bg_color, fg=fg_color)
        label.grid(row=row, column=column, padx=5, pady=5)
        column += 1
        if column == 4:
            column = 0
            row += 1
            
# Add the Done button to finish n[k] input
    done_button = tk.Button(n_frame, text="Show Initial Configuration of Blocks",font=bold_font, command=lambda: hide_n_values(b_counts), bg=button_color, fg=button_text_color)
    done_button.grid(row=row + 1, column=0, columnspan=4, pady=10)

    n_frame.grid(row=5, columnspan=2)  # Show the n_frame

# Function to hide the n values and store them for optimization
def hide_n_values(b_counts):
    n_frame.grid_remove()  # Hide the n_frame
    for widget in n_frame.winfo_children():
        widget.grid_remove()  # Hide the widgets
    
# Store n[k] values in a global variable for use in optimization
    global n_values
    n_values = [b_counts.get(k, 0) for k in range(len(b_counts))]

    # Show pictorial representation of the matrix
    T = int(t_entry.get())
    S = int(s_entry.get())
    show_pictorial_representation(T, S, initial=True)

def show_pictorial_representation(T, S, initial=False, final_data=None):
    if initial:
        frame = matrix_frame
    else:
        frame = final_matrix_frame

    for widget in frame.winfo_children():
        widget.destroy() # Clear previous matrix representations
     
# Create and place the shunter label
    shunter_label = tk.Label(frame, text="S", borderwidth=2, relief="solid", width=6, height=2, font=("Arial", 7, "bold"), bg=error_color, fg=button_text_color)
    shunter_label.grid(row=T, column=0, padx=5, pady=5, sticky="w")

    for i in range(T):
        track_label = tk.Label(frame, text=f"Track {i+1}",font=("Arial", 10, "bold"), bg=bg_color, fg=fg_color)
        track_label.grid(row=T-i, column=S+1, padx=5, pady=5, sticky="w")
        
        for j in range(S-1, -1, -1):
            if initial:
                b_value = clustered_matrix[i][j]
                box_label = tk.Label(frame, text=f"Block {b_value}" if b_value else "",font=("Arial", 10, "bold"), borderwidth=2, relief="solid", width=7, height=1, bg=highlight_color if b_value else "white")
                box_label.grid(row=T-i, column=S-j, padx=2, pady=2)
            else:
                b_value = final_data.get((i, j), "")
                box_label = tk.Label(frame, text=f"Block {b_value}" if b_value is not None else "",font=("Arial", 10, "bold"), borderwidth=2, relief="solid", width=7, height=1, bg=success_color if b_value or b_value == 0 else "white")
                box_label.grid(row=T-i, column=S-j, padx=2, pady=2)
                
        if initial and i == 1:
            arrow_label = tk.Label(frame, text="← Pull", font=("Arial", 8), bg=bg_color, fg=fg_color)
            arrow_label.grid(row=T-i, column=0, padx=2, pady=2)

    if initial:
        label = tk.Label(frame, text="Initial Configuration", font=("Arial", 10, "bold"), bg=bg_color, fg=fg_color)
    else:
        label = tk.Label(frame, text="Final Configuration", font=("Arial", 10, "bold"), bg=bg_color, fg=fg_color)

    label.grid(row=T+2, columnspan=S+2, pady=10)
    frame.grid(row=6, columnspan=2, padx=10, pady=10)
      
# If it's the final configuration, add the building block track
    if not initial:
        sorted_block_IDs = sorted((b_value for b_value in final_data.values() if b_value is not None), reverse=True)
        building_block_frame = tk.Frame(frame, bg=bg_color)
        building_block_frame.grid(row=T+3, column=1, columnspan=S+2, padx=10, pady=10, sticky="w")

# Add the "Building block" label with an arrow pointing to the right
        label_frame = tk.Frame(frame, bg=bg_color)
        label_frame.grid(row=T+3, column=0, padx=10, pady=10, sticky="w")

        label = tk.Label(label_frame, text="Building block (Track 0) →", font=("Arial", 10, "bold"), bg=bg_color, fg=fg_color)
        label.grid(row=0, column=0, padx=2, pady=2)

        for idx, b_value in enumerate(sorted_block_IDs):
            box_label = tk.Label(building_block_frame, text=f"{b_value}",font=("Arial", 10, "bold"), borderwidth=2, relief="solid", width=5, height=1, bg=success_color)
            box_label.grid(row=0, column=2*idx, padx=0, pady=2)

            if idx < len(sorted_block_IDs) - 1:
                dash_label = tk.Label(building_block_frame, text="->", bg=bg_color, fg=fg_color)
                dash_label.grid(row=0, column=2*idx+1, padx=0, pady=2)

# Function to highlight mandatory fields if they are empty
def highlight_mandatory_fields():
    entries = [t_entry, s_entry, b_entry, m_entry]
    filled = True
    for entry in entries:
        if not entry.get():
            entry.config(bg=input_error_color)
            filled = False
        else:
            entry.config(bg="white")
    return filled
def run_optimization():
    if not highlight_mandatory_fields():
        messagebox.showerror("Error", "Please fill in all mandatory fields.")
        return

    try:
        T = int(t_entry.get())
        S = int(s_entry.get())
        B = int(b_entry.get())
        M = int(m_entry.get())
    except ValueError:
        messagebox.showerror("Error", "Please enter valid integers for T, S, B, and M.")
        return

# Ensure clustered_matrix is initialized
    if 'clustered_matrix' not in globals():
        messagebox.showerror("Error", "Please cluster wagons before running optimization.")
        return

    constraints = []
    initial_data = {}

# Populate constraints and initial_data from clustered_matrix
    for i in range(T):
        for j in range(S):
            entry = clustered_matrix[i][j]  # Access the entry directly
            if entry != "":
                try:
                    b_value = int(entry)
                    x_value = 1  # Assuming x_val is 1 for this example
                    constraints.append((i, j, b_value, 0, x_value))
                    initial_data[(i, j)] = b_value
                except ValueError:
                    messagebox.showerror("Error", f"Please enter valid integers for X[{i}][{j}] and b values.")
                    return

# Compute n_values based on clustered_matrix
    unique_b_values = set()
    b_counts = Counter()
    for i in range(T):
        for j in range(S):
            if clustered_matrix[i][j] != "":
                b_value = int(clustered_matrix[i][j])
                unique_b_values.add(b_value)
                b_counts[b_value] += 1

    global n_values
    n_values = [b_counts.get(k, 0) for k in range(B)]

    results = optimize_model(T, S, B, M, constraints)

    if results:
        display_results(results, T, S, B, M)
    else:
        messagebox.showerror("Error", "Optimization failed.")

def display_results(results, T, S, B, M):
    M = len(results['X'][0][0][0])  # Use the length of one of the arrays to determine the correct M

    result_window = tk.Toplevel(root)
    result_window.title("Optimization Results")
    result_window.configure(bg=bg_color)

    result_text = f"Time required to rearrange blocks on stub yard: {results['ObjectiveValue']} Seconds\n\n"
    
    result_text += "Shunter Movements:\n\n"
    steps = []

    for m in range(M):
        for b in range(B):
            for t in range(T):
                for s in range(S):
                    if results['w'][t][s][b][m] == 1:
                        found_match = False
                        for t2 in range(T):
                            for s2 in range(S):
                                if results['z'][t2][s2][b][m] == 1:
                                    result_text += f"Step {m+1}: Shunter pulls Block with block id {b} from Track {t+1}, Slot {s+1} and pushes it to Track {t2+1}, Slot {s2+1}\n"
                                    steps.append((t, s, t2, s2, b, m))
                                    found_match = True
                                    break
                            if found_match:
                                break
   
    result_label = tk.Label(result_window, text=result_text, bg=bg_color, fg=fg_color, justify=tk.LEFT)
    result_label.pack(padx=10, pady=10)

    final_frame = tk.Frame(result_window, bg=bg_color)
    final_frame.pack(padx=10, pady=10)

    final_data = {(i, j): None for i in range(T) for j in range(S)}
    for i in range(T):
        for j in range(S):
            for k in range(B):
                if results['X'][i][j][k][M-1] == 1:
                    final_data[(i, j)] = k

    show_pictorial_representation(T, S, initial=False, final_data=final_data)
    results['steps'] = steps

def optimize_model(T, S, B, M, constraints):
    # Function to create the model with binary variables
    def create_binary_model(M):
        model = gp.Model("optimization_problem")
        
        X = {}
        w = {}
        z = {}
        y = {}
        
        for t in range(T):
            X[t] = {}
            for s in range(S):
                X[t][s] = {}
                for b in range(B):
                    X[t][s][b] = model.addVars(M, vtype=GRB.BINARY, name=f"X_{t}_{s}_{b}")

        for t in range(T):
            w[t] = {}
            for s in range(S):
                w[t][s] = {}
                for b in range(B):
                    w[t][s][b] = model.addVars(M, vtype=GRB.CONTINUOUS, name=f"w_{t}_{s}_{b}")

        for t in range(T):
            z[t] = {}
            for s in range(S):
                z[t][s] = {}
                for b in range(B):
                    z[t][s][b] = model.addVars(M, vtype=GRB.CONTINUOUS, name=f"z_{t}_{s}_{b}")
                    
        for t in range(T):
            y[t] = {}
            for t1 in range(T):
                 y[t][t1] = {}
                 for s in range(S):
                    y[t][t1][s] = {}
                    for b in range(B):
                        y[t][t1][s][b] = model.addVars(M, vtype=GRB.BINARY, name=f"y_{t}_{t1}_{s}_{b}")
                        
        print("e_values:", e_values)                
        df1= pd.read_excel('Flat_Yard_Marshalling_Problem_Parameters.xlsx', 'A')  
        my_tuple = [tuple(x) for x in df1.values]
        o =dict((tuple((a, b)), c) for a,b,c in df1.values)

        # Read the Excel file
        df1 = pd.read_excel('Flat_Yard_Marshalling_Problem_Parameters.xlsx', sheet_name='B')
        my_tuple = [tuple(x) for x in df1.values]
        D = dict((a, c) for a, c in df1.values)

        Obj1 = gp.quicksum(((D[t]/(2*(16/3600))) + 30 + 1.08 *o.get((0, t), 0))*w[t][s][b][0] 
                               for t in range(T)
                               for s in range(S)
                               for b in range(B)
                          )
        Obj2 = gp.quicksum((D[t]/(2*(16/3600)) + 30 + 1.15 * e_values[b]) * w[t][s][b][m] 
                               for t in range(T)
                               for s in range(S)
                               for b in range(B)
                               for m in range(1,M-1)
                          )
        Obj3 = gp.quicksum(1.08 *o.get((t, t1), 0) * y[t][t1][s][b][m]  
                               for t in range(T)
                               for t1 in range(T)
                               for s in range(S)
                               for b in range(B)
                               for m in range(1,M-1)
                          )
        Obj4 = gp.quicksum((D[t]/(2*(16/3600)) + 30 + 1.15 * e_values[b]) * z[t][s][b][m]
                               for t in range(T)
                               for s in range(S)
                               for b in range(B)
                               for m in range(1,M-1)
                          )
        Obj5 = gp.quicksum(((D[t]/(2*(16/3600))) + 30 + 1.08 *o.get((t, 0), 0)) * z[t][s][b][M-1]  
                               for t in range(T)
                               for s in range(S)
                               for b in range(B)
                          ) 
        # Objective
        OBJ = Obj1 + Obj2 + Obj3 + Obj4 + Obj5
        model.setObjective(OBJ, GRB.MINIMIZE)

        # Add constraints
        for t in range(T):
            for t1 in range(T):
                for s in range(S):
                    for b in range(B):
                        for m in range(1,M-1):
                            model.addConstr(y[t][t1][s][b][m] >= w[t][s][b][m] + z[t1][s][b][m] - 1, f"Constraint00")
 
        for i, j, b_value, t_index, x_value in constraints:
            model.addConstr(X[i][j][b_value][t_index] == x_value)

        for k in range(B):
            for m in range(M):
                con1 = sum(X[i][j][k][m] for i in range(T) for j in range(S))
                model.addConstr(con1 == n_values[k], f"Constraint1a_{k}_{m}")
        
        # Constraint 1:
        for b in range(B):
            for m in range(M-1):
                con1 = sum(w[t][s][b][m] for t in range(T) for s in range(S))
                con2 = sum(z[t][s][b][m] for t in range(T) for s in range(S))
                model.addConstr(con1 == con2, f"Constraint1")

        # Constraint 2:
        for m in range(M-1):
            con1 = sum(w[t][s][b][m] for t in range(T) for s in range(S) for b in range(B))
            model.addConstr(con1 <= 1, f"Constraint2a")

        for m in range(M-1):
            con1 = sum(z[t][s][b][m] for t in range(T) for s in range(S) for b in range(B))
            model.addConstr(con1 <= 1, f"Constraint2b")

        # Constraint 3:
        for m in range(M-2):
            con1 = sum(w[t][s][b][m+1] for t in range(T) for s in range(S) for b in range(B))
            con2 = sum(w[t][s][b][m] for t in range(T) for s in range(S) for b in range(B))
            model.addConstr(con1 <= con2, f"Constraint3")

        # Constraint 4:
        for t in range(T):
            for m in range(1,M-1):
                con1 = sum(X[t][0][b][m] for b in range(B))
                con2 = sum(z[t][0][b][m] for b in range(B))
                model.addConstr(con1 + con2 <= 1, f"Constraint4")

        # Constraint 5:
        for t in range(T):
            for s in range(S - 1):
                for b in range(B):
                    con1 = sum(X[t][s + 1][k1][M - 1] for k1 in range(b, B))
                    con2 = sum(X[t][s][k1][M - 1] for k1 in range(b, B))
                    model.addConstr(con1 <= con2, f"Constraint5")

        # Constraint 6:
        for t in range(T):
            for s in range(S):
                for b in range(B):
                    for m in range(M-1):
                        model.addConstr(X[t][s][b][m] + z[t][s][b][m] == X[t][s][b][m+1] + w[t][s][b][m], f"Constraint6")

        # Constraint 7:
        for t in range(T):
            for s in range(S-1):
                for m in range(M-1):
                    con1 = sum(X[t][s + 1][b][m] for b in range(B))
                    con2 = sum(w[t][s][b][m] for b in range(B))
                    con3 = sum(z[t][s + 1][b][m] for b in range(B))
                    con4 = sum(X[t][s][b][m] for b in range(B)) 
                    model.addConstr(con1 + con2 + con3 <= con4, f"Constraint7")

        # Constraint 8:
        for t in range(T):
            for s in range(S-1):
                for m in range(M-1):
                    con1 = sum(X[t][s][b][m] for b in range(B))
                    con2 = sum(X[t][s+1][b][m] for b in range(B))
                    con3 = sum(w[t][s][b][m] for b in range(B))
                    con4 = sum(z[t][s+1][b][m] for b in range(B)) 
                    con5 = sum(z[t][s][b][m] for b in range(B))
                    model.addConstr(con1 + con2 + con3 + con4 + con5 <= 2 , f"Constraint8")

        # Constraint 9:
        for t in range(T):
            for m in range(M-2):
                con1 = sum(z[t][s][b][m] for s in range(S) for b in range(B))
                con2 = sum(w[t][s][b][m+1] for s in range(S) for b in range(B))
                model.addConstr(con1 + con2 <= 1 , f"Constraint9")

        # Constraint 10:
        for t in range(T):
            for b in range(B):
                for m in range(M-2):
                    con1 = sum(w[t][s][b][m] for s in range(S))
                    con2 = sum(z[t][s][b][m+1] for s in range(S))
                    model.addConstr(con1 + con2 <= 1 , f"Constraint10")

        # Constraint 11:
        for t in range(T):
            for s in range(S):
                model.addConstr(w[t][s][0][M-2] == 0 , f"Constraint11a")
                model.addConstr(z[t][s][0][M-2] == 0 , f"Constraint11a")

        # Constraint 12:
        for t in range(T):
            for b in range(1,B):
                model.addConstr(w[t][0][0][M-2] == 0 , f"Constraint12")

        return model, X, w, z, y

   # Solve LP relaxation
    model, X, w, z, y = create_binary_model(M)
    for t in range(T):
        for s in range(S):
            for b in range(B):
                for m in range(M):
                    X[t][s][b][m].vtype = GRB.CONTINUOUS
                    w[t][s][b][m].vtype = GRB.CONTINUOUS
                    z[t][s][b][m].vtype = GRB.CONTINUOUS

    for t in range(T):
        for t1 in range(T):
            for s in range(S):
                for b in range(B):
                    for m in range(M):
                        y[t][t1][s][b][m].vtype = GRB.CONTINUOUS
    model.optimize()
    if model.Status != GRB.OPTIMAL:
        print("LP relaxation did not solve to optimality.")
        return None

    # Get the value of z from the LP relaxation
    z_value = sum(z[t][s][b][m].X for t in range(T) for s in range(S) for b in range(B) for m in range(M))
    print(f"Sum of z values: {z_value}")

    # Set initial T value
    M = int(z_value) + 1

    # Solve original problem with binary variables
    while True:
        model, X, w, z, y = create_binary_model(M)
        model.optimize()
        if model.Status == GRB.OPTIMAL:
            results = {
                "ObjectiveValue": model.ObjVal,
                "w": {i: {j: {k: {t: w[i][j][k][t].x for t in range(M)} for k in range(B)} for j in range(S)} for i in range(T)},
                "z": {i: {j: {k: {t: z[i][j][k][t].x for t in range(M)} for k in range(B)} for j in range(S)} for i in range(T)},
                "X": {i: {j: {k: {t: X[i][j][k][t].x for t in range(M)} for k in range(B)} for j in range(S)} for i in range(T)},
                "y" : {s: {s1: {h: {p: {t: y[s][s1][h][p][t].x for t in range(M)} for p in range(B)} for h in range(S)} for s1 in range(T)} for s in range(T)}
            }
             # Print ObjectiveValue
            print("ObjectiveValue:", results["ObjectiveValue"])

            # Print w values
            print("w values:")
            for i in results["w"]:
                for j in results["w"][i]:
                    for k in results["w"][i][j]:
                        for t in results["w"][i][j][k]:
                            if results['w'][i][j][k][t] != 0:
                                print(f"w[{i}][{j}][{k}][{t}] = {results['w'][i][j][k][t]}")

            # Print z values
            print("z values:")
            for i in results["z"]:
                for j in results["z"][i]:
                    for k in results["z"][i][j]:
                        for t in results["z"][i][j][k]:
                            if results['z'][i][j][k][t] != 0:
                                print(f"z[{i}][{j}][{k}][{t}] = {results['z'][i][j][k][t]}")

            # Print X values
            print("X values:")
            for i in results["X"]:
                for j in results["X"][i]:
                    for k in results["X"][i][j]:
                        for t in results["X"][i][j][k]:
                            if results['X'][i][j][k][t] != 0:
                                print(f"X[{i}][{j}][{k}][{t}] = {results['X'][i][j][k][t]}")
                                
            
            return results
        else:
            print(f"Solution infeasible for M={M}. Incrementing M and trying again.")
            M += 1
    return None


# Create the main application window    
root = tk.Tk()
root.title("Flat Yard Marshalling Software")
root.configure(bg=bg_color)

# Title Label
tk.Label(root, text="Number of Tracks (T):", bg=bg_color, fg=fg_color, font=bold_font).grid(row=0, column=0, padx=10, pady=5)
# S Input
t_entry = tk.Entry(root)
t_entry.grid(row=0, column=1, padx=10, pady=5)

# H Input
tk.Label(root, text="Number of Slots per track (S):", bg=bg_color, fg=fg_color, font=bold_font).grid(row=1, column=0, padx=10, pady=5)
s_entry = tk.Entry(root)
s_entry.grid(row=1, column=1, padx=10, pady=5)

# P Input
tk.Label(root, text="Block IDs of Wagon (B):", bg=bg_color, fg=fg_color, font=bold_font).grid(row=2, column=0, padx=10, pady=5)
b_entry = tk.Entry(root)
b_entry.grid(row=2, column=1, padx=10, pady=5)

# T Input
tk.Label(root, text="Time Points (M):", bg=bg_color, fg=fg_color, font=bold_font).grid(row=3, column=0, padx=10, pady=5)
m_entry = tk.Entry(root)
m_entry.grid(row=3, column=1, padx=10, pady=5)

generate_button = tk.Button(root, text="Insert Initial Configuration of Wagons", font=bold_font, command=lambda: create_dynamic_inputs(int(t_entry.get()), int(s_entry.get())), bg=button_color, fg=button_text_color)
generate_button.grid(row=4, columnspan=2, pady=10)

# Frame for dynamic entry fields
dynamic_frame = tk.Frame(root, bg=bg_color)
dynamic_frame.grid(row=5, columnspan=2)

n_frame = tk.Frame(root, bg=bg_color)

# Run Button
run_button = tk.Button(root, text="Solve Problem", font=bold_font, command=run_optimization, bg=button_color, fg=button_text_color)
run_button.grid(row=7, columnspan=2, pady=10)

# Create frames for the matrix representations
matrix_frame = tk.Frame(root, bg=bg_color)
matrix_frame.grid(row=6, columnspan=2, padx=10, pady=10, sticky="nsew")

final_matrix_frame = tk.Frame(root, bg=bg_color)
final_matrix_frame.grid(row=6, column=2, padx=10, pady=10, sticky="nsew")

root.grid_rowconfigure(6, weight=1)
root.grid_columnconfigure(1, weight=1)

dynamic_entries = {}
n_values = []

# Main loop
root.mainloop()